---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

## Import the raw data

In [1]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
test = df[:]
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

### Helper Functions

In [2]:
def count_empties(this_df):
    counter = 0
    for each in this_df:
        if (each == []) or (type(each) == str):
            counter+=1
    print('Dates Missing {}'.format(counter))
    found = this_df.shape[0] - counter
    print('Dates Found {}'.format(found))

In [3]:
def merge_found(x,y):
    if x == []:
        return y
    else:
        return x 

In [4]:
def not_null(a,b):
    if a.isnull():
        return b
    else:
        return a

In [5]:
def merge_missing(a,b):
    if type(a) == list:
        return a
    else:
        return b

In [6]:
def insert_missing(got_it,missing):
    if len(got_it) == 0:
        return missing
        return got_it

In [7]:
def get_good_index(temp_df):
    cnt = 0
    good_index = []
    for entry,idx in zip(temp_df,temp_df.index.tolist()):
        if type(entry) == list:
            if len(entry) > 0:
                print(entry)
                cnt += 1
                good_index.append(idx)
    return(good_index)

### Create Final DF for Sorting

In [8]:
final_df = df[:]

### First Iteration: Find the Month / Day / Year formats

In [9]:
# m(m)/d(d)/yy(yy), m(m)/d(d)/yy
# numbers only: 125
nums_only = r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}'
nums_only_df = final_df.str.findall(nums_only)

In [10]:
#nums_only_df[:50]

In [11]:
#count_empties(nums_only_df)

In [12]:
# add nums_only dates to final_DF
nums_only_good_index = get_good_index(nums_only_df)
final_df[nums_only_good_index] = nums_only_df[nums_only_good_index]

['03/25/93']
['6/18/85']
['7/8/71']
['9/27/75']
['2/6/96']
['7/06/79']
['5/18/78']
['10/24/89']
['3/7/86']
['4/10/71']
['5/11/85']
['4/09/75']
['8/01/98']
['1/26/72']
['5/24/1990']
['1/25/2011']
['4/12/82']
['10/13/1976']
['4/24/98']
['5/21/77']
['7/21/98']
['10/21/79']
['3/03/90']
['2/11/76']
['07/25/1984']
['4-13-82']
['9/22/89']
['9/02/76']
['9/12/71']
['10/24/86']
['03/31/1985']
['7/20/72']
['4/12/87']
['06/20/91']
['5/12/2012']
['3/15/83']
['2/14/73']
['5/24/88']
['7/27/1986']
['1-14-81']
['7-29-75']
['6/24/87']
['8/14/94']
['4/13/2002']
['8/16/82']
['2/15/1998']
['7/15/91']
['06/12/94']
['9/17/84']
['2/28/75']
['11/22/75']
['5/24/91']
['6/13/92']
['7/11/71']
['12/26/86']
['10/11/1987']
['3/14/95']
['12/01/73']
['12/5/2010']
['08/20/1982']
['7/24/95']
['8/06/83']
['02/22/92']
['6/28/87']
['07/29/1994']
['08/11/78']
['10/29/91']
['7/6/91']
['1/21/87']
['11/3/1985']
['7/04/82']
['4-13-89']
['7/11/77', '9/36/308']
['4/12/74']
['09/19/81']
['9/6/79']
['12/5/87']
['01/05/1999']
['4/22/

In [13]:
# check results
#final_df

### Second Iteration: Find Alpha Numeric Dates

In [14]:
# to grab dates with Month at the beginning
#alpha_present = r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2}, )?\d{4}'
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Sept|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_present_df = final_df.str.findall(alpha_v2)
#count_empties(alpha_present_df)

In [15]:
#add alphanumeric dates to final_DF
alpha_present_good_index = get_good_index(alpha_present_df)
final_df[alpha_present_good_index] = alpha_present_df[alpha_present_good_index]

['24 Jan 2001']
['10 Sep 2004']
['26 May 1982']
['28 June 2002']
['06 May 1972']
['25 Oct 1987']
['14 Oct 1996']
['30 Nov 2007']
['28 June 1994']
['14 Jan 1981']
['10 Oct 1985']
['11 February 1985']
['10 Feb 1983']
['05 Feb 1992']
['21 Oct 2012']
['14 Feb 1995']
['30 May 2016']
['22 Jan 1996']
['14 Oct 1992']
['06 Oct 2003']
['18 Oct 1999']
['11 Nov 2004']
['30 May 2001']
['02 Feb 1978']
['09 Sep 1989']
['12 March 1980']
['22 June 1990']
['28 Sep 2015']
['13 Jan 1972']
['06 Mar 1974']
['10 Oct 1974']
['26 May 1974']
['10 Feb 1990']
['23 Aug 2000']
['26 May 2001']
['21 Oct 2007']
['19 Oct 2016']
['05 Mar 1974']
['29 Jan 1994']
['21 Oct 1978']
['18 August 1975']
['11 Nov 1996']
['01 Oct 1979']
['13 Oct 1986']
['21 Oct 1995']
['24 Jan 2011']
['04 Oct 1972']
['23 Aug 1993']
['18 Oct 2006']
['04 Dec 1988']
['21 Oct 1983']
['26 May 2010']
['18 Jan 1990']
['15 Jun 1985']
['10 Dec 1982']
['09 Dec 1988']
['18 August 1995']
['13 June 1974']
['26 May 2008']
['11 Nov 2002']
['17 Aug 1985']
['13 Oc

### Find entries with Month/Year m(m)/yyyy format

In [16]:
# regex for month/year
month_year = r'\d{,2}[/]+\d{2,4}'
month_year_df = final_df.str.findall(month_year)

In [17]:
# indices of newly found entries
month_year_good_index = get_good_index(month_year_df)
final_df[month_year_good_index] = month_year_df[month_year_good_index]

['6/1998']
['6/2005']
['10/1973']
['9/2005']
['03/1980']
['12/2005']
['5/1987']
['5/2004']
['8/1974']
['3/1986']
['10/1997']
['3/1993']
['3/1981']
['9/2003']
['10/1993']
['1/1983']
['7/1994']
['12/2008']
['10/1980']
['8/2003']
['12/1975']
['11/2010']
['7/1997']
['7/2014']
['9/2001']
['8/1986']
['1/1978']
['9/1975']
['1/2009']
['5/1995']
['8/1989']
['11/2000']
['06/1973']
['2/1999']
['6/2001']
['12/1978']
['8/2009']
['7/1973']
['1/2014']
['09/1975']
['12/2012']
['5/1999']
['7/1989']
['7/2009']
['11/1998']
['3/1995']
['2/2009']
['08/1988']
['4/2007']
['5/2000']
['4/2012']
['10/2001']
['2/1977']
['8/2008']
['2/1983']
['5/1979']
['1/1992']
['11/2008']
['12/2014']
['4/1974']
['10/1981']
['10/1986']
['03/1973']
['12/1994']
['8/1999']
['10/2010']
['10/1994']
['01/2007']
['8/2010']
['03/1990']
['11/2016']
['4/2004']
['2/1973']
['7/1987']
['8/2000']
['8/1975']
['5/1977']
['3/2000']
['9/1984']
['4/1973']
['12/1986']
['4/1979']
['7/2004']
['11/1984']
['5/2016']
['12/2007']
['11/1982']
['06/1981']

In [18]:
#count_empties(final_df)

### In Final DF, Fined Entires with Only Year Values

In [19]:
# regex for year only
only_year = r'[a-zA-Z]?(\d{4})'
only_year_df = final_df.str.findall(only_year)

In [20]:
# indices of newly found entries
only_year_good_index = get_good_index(only_year_df)

['1984']
['2000']
['2001']
['1982']
['1998']
['2012']
['1991']
['1988']
['2014']
['2016']
['1976']
['1981']
['2011']
['1997']
['2003']
['1983']
['1999']
['2010']
['1975']
['1972']
['2015']
['1989']
['1994']
['1993']
['1996']
['2013']
['1974']
['1990']
['1995']
['2004']
['1987']
['1973']
['1992']
['1977']
['1985']
['2007']
['2009']
['1986']
['1978']
['2002']
['1979']
['2006']
['2008']
['2005']
['1980']


In [21]:
# Insert newly found entries for month/year into Combined DF, Final.
final_df[only_year_good_index] = only_year_df[only_year_good_index]

In [22]:
#count_empties(final_df)

In [23]:
#final_df[455:]

## data cleaning: fix errors and convert to standard format

In [24]:
import dateutil

#### Fix Errors in Alpha_Present_DF

In [25]:
idx = 228
for each in final_df[228:343]:
    temp = each[0].strip(" ")
    if "Janaury" in temp:
        temp = temp.replace("Janaury","Jan")
    elif "Decemeber" in temp:
        temp = temp.replace("Decemeber","Dec")
    temp2 = "1 "+ temp
    final_df.loc[idx] = [temp2]
#    print(temp2)
    idx+=1

In [26]:
'''
idx = 228
for each in final_df[228:343]:
    print(idx,each)
    idx+=1
'''

'\nidx = 228\nfor each in final_df[228:343]:\n    print(idx,each)\n    idx+=1\n'

In [27]:
# check results
#for each in final_df.loc[340:345]:
#    print(dateutil.parser.parse(each[0]))

#### Fix Errors in Month/Year

In [28]:
for each in final_df[343:455]:
    temp = each[0].strip(" ")
    day_year = temp[:-4] + "1/" + temp[-4:]
#    print(day_year)

In [29]:
# 343 - 455
idx = 343
for each in final_df[343:455]:
    temp = each[0].strip(" ")
    day_year = temp[:-4] + "1/" + temp[-4:]
    final_df.loc[idx] = [day_year]
    idx +=1

In [30]:
# check results
#for each in final_df.loc[[343,454]]:
#    print(dateutil.parser.parse(each[0]))

### Add day month to the year only entries

In [31]:
idx = 455
for each in final_df[455:]:
    temp = each[0].strip(" ")
    mo_day_year = "1/1/"+temp[-4:]
    final_df.loc[idx] = [mo_day_year]
#    print(mo_day_year)
    idx +=1

### Fix one random, stupid error

In [32]:
final_df[321] = '2 June, 1999'

In [33]:
date_time_equivalents = []
for idx,each in enumerate(final_df):
    temp = dateutil.parser.parse(each[0], dayfirst=False)
    date_time_equivalents.append(temp)


In [34]:
#for each in date_time_equivalents

In [35]:
clean_df = pd.Series(date_time_equivalents)

In [36]:
'''
for i in range(500):
    print(df[i].strip("\n"))
    print(date_time_equivalents[i])
    a = (dateutil.parser.parse(final_df[i][0], dayfirst=False))
    print(a.strftime('%b %d %Y'))
    print()
'''

'\nfor i in range(500):\n    print(df[i].strip("\n"))\n    print(date_time_equivalents[i])\n    a = (dateutil.parser.parse(final_df[i][0], dayfirst=False))\n    print(a.strftime(\'%b %d %Y\'))\n    print()\n'

In [37]:
'''
for idx,each in enumerate(df):
    if "2June, 1999" in each:
        print(idx)
'''

'\nfor idx,each in enumerate(df):\n    if "2June, 1999" in each:\n        print(idx)\n'

In [38]:
#df[321]

In [39]:
#final_df[321] = '2 June, 1999'

In [40]:
#final_df[321]

In [43]:
a1 = pd.DataFrame({'date':clean_df})
a2 = a1.sort_values('date')
a2

,date
9,1971-04-10
84,1971-05-18
2,1971-07-08
53,1971-07-11
28,1971-09-12
474,1972-01-01
153,1972-01-13
13,1972-01-26
129,1972-05-06
98,1972-05-13


In [51]:
a3 = a2.reset_index()
a3

,index,date
0,9,1971-04-10
1,84,1971-05-18
2,2,1971-07-08
3,53,1971-07-11
4,28,1971-09-12
5,474,1972-01-01
6,153,1972-01-13
7,13,1972-01-26
8,129,1972-05-06
9,98,1972-05-13


In [59]:
#a3.date.value_counts()

In [44]:
ans = a2.reset_index()['index']
ans

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    243
471    208
472    139
473    320
474    383
475    286
476    244
477    480
478    431
479    279
480    198
481    381
482    463
483    366
484    439
485    255
486    401
487    475
488    257
489    152
490    235
491    464
492    253
493    231
494    427
495    141
496    186
497    161
498    413
499    321
Name: index, dtype: int64

In [50]:
#theory = a2.reset_index()['index']

405

In [56]:
#a3[a3['date'] == '2013-01-01']

,index,date
475,286,2013-01-01
476,244,2013-01-01
477,480,2013-01-01


In [60]:
def date_sorter():
    return ans
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    243
471    208
472    139
473    320
474    383
475    286
476    244
477    480
478    431
479    279
480    198
481    381
482    463
483    366
484    439
485    255
486    401
487    475
488    257
489    152
490    235
491    464
492    253
493    231
494    427
495    141
496    186
497    161
498    413
499    321
Name: index, dtype: int64